# Chapter 6 - Statistics and Linear Regressions

In this chapter we will learn to run some summary statistics and run a simple linear regression. There are many ways to run a linear regression in Python, but the package we'll be focusing on it **statsmodel**. 

***

Let's start by importing the pertinent libraries.

In [ ]:
#import standard libraries
import numpy as np
import pandas as pd
import math
import glob
from datetime import datetime
import warnings
warnings.simplefilter('ignore')

#import statistical libraries
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.sandbox.regression.predstd import wls_prediction_std

#import visualization libraries
import matplotlib.pyplot as plt

## Read in and Process Data

First we must read our data into Python. We'll be reading in the seabird breeding success datasheet and the El Niño-Southern Oscillation (ENSO) data. This is an index generated by the National Oceanographic and Atmospheric Administration (NOAA). The data is based on sea surface temperature anomalies using a 30-year period and represents a three month mean. Values can be access from the NOAA site and are updated monthly: https://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_v5.php

In [ ]:
###Read in the seabird data
bird = pd.read_csv('/content/drive/MyDrive/data/seabird_bs_may23.csv', index_col = [0])

#select out your two bird species
penguin = bird[bird['sppsite'] == 'little penguin_Oamaru'][['year_recorded','site', 'lat', 'lon', 'spp', 'bs']]
shearwater = bird[bird['sppsite'] == 'sooty shearwater_Kauwahaia'][['year_recorded','site', 'lat', 'lon', 'spp', 'bs']]

#join the data together
bird_combined = penguin.merge(shearwater, on = 'year_recorded', how = 'outer', suffixes = ('_penguin', '_shearwater'))
bird_combined

In [ ]:
###Read in the ENSO data
enso = pd.read_csv('/content/drive/MyDrive/data/ENSO_values.csv')
enso

In [ ]:
###Join the data together
#select out the DJM ENSO
enso_DJM = enso.loc[enso['Month'] == 'DJF'].drop(['Month', 'Month_num', 'Date'], axis = 1)

#join the data based on year
bird_combined = bird_combined.merge(enso_DJM, how = 'outer', left_on = 'year_recorded', right_on = 'Year')

#select out the columns we want
bird_combined = bird_combined[['Year', 'bs_penguin', 'bs_shearwater', 'ENSO']]

#remove missing values based on years we don't have breeding success for
bird_combined = bird_combined.dropna(subset = ['bs_penguin', 'bs_shearwater'], how = 'all').sort_values('Year').reset_index(drop = True)
bird_combined

## Explore the Data

### Summary Statistics

In addition to graphs and visualizing your data, it's also important to look at summary statistics to get a better understanding of it. The most basic summary statistics (mean, median, and mode) along with some other useful ones can easily be called on a **pandas** dataframe. The `agg()` function allows you to call multiple summary stats at once. 

In [ ]:
#show the mean of each column
bird_combined.mean()

In [ ]:
#how the mean and the standard deviation
bird_combined.agg(['mean', 'std'])

In [ ]:
#use the describe function to get lots of information about your data
bird_combined.describe().round(2)
#the .round() function is useful to round all numbers to a specified decimal which cleans up the output

### Outlier Detection

Sometimes the data contains outliers, data that falls well outside the rest of the data. Outliers can be useful in helping detect "bad" observations, though sometimes outliers are legitimate observations. The choice to include or remove outliers from the data is ultimately up to you. Still though it is useful to identify outliers to either be removed or further investigated. 

One method for outlier detection is to calculate percentiles, a score below which a given percentage falls. Looking at the high and low percentiles can be useful. 

In [ ]:
###Calculate percentiles
#calculate the 1st percentile
percentile1 = np.nanpercentile(bird_combined['bs_penguin'], 1)
print('1st percentile:', percentile1)

#calculate the 99th percentile
percentile99 = np.nanpercentile(bird_combined['bs_penguin'], 99)
print('99th percentile:', percentile1)

#filter the data outside these bounds
bird_combined[(bird_combined['bs_penguin'] <= percentile1) | (bird_combined['bs_penguin'] >= percentile99)]

###How many outliers are there below the 1st or about the 99th percentile?

Another useful metric is a z-score. The z-score measures how many standard deviations a data point is from the mean in a distribution. Typically an outlier is outside 3 standard deviations (99.7% of data)

<div>
<center><img src="https://i.insider.com/546e68776bb3f74f68b7d0ba?width=769&format=jpeg" width="500"/></center>
</div>

The z-score is calculated by taking each observation (x) minus the mean ($\mu$) divided by the standard deviation ($\sigma$): 

\begin{gather*}
Z = \frac{x-\mu}{\sigma}\\\
\end{gather*}

In [ ]:
###Calculate z-score
mean = bird_combined['bs_shearwater'].mean()
sd = bird_combined['bs_shearwater'].std()

bird_combined['bs_shearwater_zscore'] = (bird_combined['bs_shearwater'] - mean) / sd

#filter out data outside two standard deviations
bird_combined[np.abs(bird_combined['bs_shearwater_zscore']) > 2]

#####How many outliers do you have here? 

### Normalization

Sometimes it's necessary to normalize data. Normalization data typically transforms it to be between 0-1 and allows you to make relative comparisons. For this example, breeding success is strongly related to how many chicks a species can have. So by transforming the data to be between 0-1, the breeding success between birds that have different clutch sizes won't be a problem anymore. 

Data can be normalized by taking each observation minus the minimum value divided by the maximum minus the minimum: 

\begin{gather*}
X_{new} = \frac{X - X_{min}}{X_{max} - X_{min}}\\\
\end{gather*}

In [ ]:
###Normalize shearwater breeding successes
#calculate minum and maximum
min1 = bird_combined['bs_shearwater'].min()
max1 = bird_combined['bs_shearwater'].max()

#normalize the breeding success for shearwaters
bird_combined['bs_shearwater_norm'] = (bird_combined['bs_shearwater'] - min1) / (max1 - min1)

#look at histogram of normalized data
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (12,4))

ax[0].hist(bird_combined['bs_shearwater'], bins = 10, edgecolor = 'k', lw = 1)
ax[0].set(xlabel = 'Breeding Success', ylabel = 'Count',
          title = 'Breeding Success of Shearwater')

ax[1].hist(bird_combined['bs_shearwater_norm'], bins = 10, edgecolor = 'k', lw = 1)
ax[1].set(xlabel = 'Normalized Breeding Success', ylabel = 'Count',
          title = 'Normalized Breeding Success of Shearwater')

Notice that the distribution doesn't change but the values on the x-axis shift from 0-1. You can also easily normalize the data in one line too.

In [ ]:
#normalize the breeding success for penguins
bird_combined['bs_penguin_norm'] = (bird_combined['bs_penguin'] - bird_combined['bs_penguin'].min()) / (bird_combined['bs_penguin'].max()- bird_combined['bs_penguin'].min())

bird_combined.head()

In [ ]:
#####TASK#####
#plot normalized breeding success as time series for both species in a single plot 

## Linear Regression

There are several libraries you can use to run a linear regression. **Sci-Kit Learn** and **statsmodels** are two commons ones. The output values (i.e. slope and intercept) will be the same, but the syntax and actual output from the two different models will be different. As scientists, we prefer to use **statsmodels** because the output gives you more information and information necessary for reporting results to the scientific community.

In [ ]:
###Plot of ENSO and breeding success
#create figure
fig, ax = plt.subplots(1, 1, figsize = (8, 6))

#add points for both species
ax.scatter(bird_combined['ENSO'], bird_combined['bs_penguin_norm'], label = 'Penguin', marker = 'o', c = 'tab:blue')
ax.scatter(bird_combined['ENSO'], bird_combined['bs_shearwater_norm'], label = 'Shearwater', marker = '^', c = 'tab:orange')

#add labels and title
ax.set(xlabel = 'ENSO Value', ylabel = 'Breeding Success', title = "ENSO Values vs Breeding Success")

#add legend
ax.legend()

#add lines for 0 and 0.5 (middle values for ENSO and breeding success respectively)
ax.axvline(0, linestyle = 'dashed', lw = 1, color = 'grey', zorder = 0)
ax.axhline(0.5, linestyle = 'dashed', lw = 1, color = 'grey', zorder = 0)

In [ ]:
###Run the linear regression for one species
#select out x and y
X = bird_combined['ENSO']
Y = bird_combined['bs_penguin_norm']

#add a condstant for the regression
X = sm.add_constant(X)

#run the model
model_penguin = sm.OLS(Y, X, missing = 'drop').fit()

#look at the model output
model_penguin.summary()

In [ ]:
#####TASK#####
#run a regression for the second species

In [ ]:
#you can also extract out some of the necessary values
slope = model_penguin.params[1]
intercept = model_penguin.params[0]

#and extract out model summary values 
p_val = model_penguin.pvalues[1]
r_squared = model_penguin.rsquared

#calculate confidence intervals
_, upper,lower = wls_prediction_std(model_penguin)

In [ ]:
#use the output to calculate the predicted values
y_pred = slope * X['ENSO'] + intercept

#join the predicted values back to the full data
bird_combined['bs_penguin_pred'] = y_pred

#add the upper and lower bounds
bird_combined['penguin_upper_ci'] = upper
bird_combined['penguin_lower_ci'] = lower

bird_combined

In [ ]:
###Create Scatter Plot with regression line
#create plot
fig, ax = plt.subplots(1, 1, figsize = (8, 6))

#add the data
ax.scatter(bird_combined['ENSO'], bird_combined['bs_penguin_norm'])

#add the regression line
ax.plot(bird_combined.sort_values('bs_penguin_pred')['ENSO'], bird_combined.sort_values('bs_penguin_pred')['bs_penguin_pred'], c = 'red', linestyle = 'dashed')

#add the error bars for the regression line
ax.fill_between(bird_combined.sort_values('bs_penguin_pred')['ENSO'], 
                bird_combined.sort_values('bs_penguin_pred')['penguin_upper_ci'], 
                bird_combined.sort_values('bs_penguin_pred')['penguin_lower_ci'], 
                interpolate = True, color = 'tab:blue', alpha = 0.2)

# plt.plot(corm_joined.sort_values('predicted_bs')['ENSO'], corm_joined.sort_values('predicted_bs')['upper_ci'], linestyle = 'dashed')
# plt.plot(corm_joined.sort_values('predicted_bs')['ENSO'], corm_joined.sort_values('predicted_bs')['lower_ci'], linestyle = 'dashed')

#add the rsquared and p-value
ax.text(0.95, 0.93, u"R\u00b2: {:0.3f}".format(r_squared), horizontalalignment = 'right', transform=plt.gca().transAxes)
ax.text(0.95, 0.89, u"p-value: {:0.3f}".format(p_val), horizontalalignment = 'right', transform=plt.gca().transAxes)

#add labels and title
ax.set(xlabel = 'ENSO Value', ylabel = 'Breeding Success', title = "ENSO Values vs Breeding Success of Brandt's Commorant")


In [ ]:
#####TASK#####
#plot the data for both species with lines (no confidence intervals) on the same plot

## Second-Order Regression

It is also usually a good idea to try polynomial regressions (ie. fitting a quadratic equation for a second order model). You can try more complicated models, but it's typically best to use the simpliest model that fits the data.

In [ ]:
#select out x and y first removing nas
X = bird_combined.dropna(subset = 'bs_penguin_norm')['ENSO']
Y = bird_combined.dropna(subset = 'bs_penguin_norm')['bs_penguin_norm']

#prepare x variable to be transformed
inds = X.ravel().argsort()  # Sort x values and get index
x = X.ravel()[inds].reshape(-1,1)

#transform the x-axis to a second degree polynomial
polynomial_features = PolynomialFeatures(degree=2)
xp = polynomial_features.fit_transform(x)

#fit the model
model = sm.OLS(Y, xp).fit()

#extract out model parameters we want
p_val = model.pvalues[1]
r_squared = model.rsquared

#look at the model output
model.summary()

In [ ]:
#use the built in function to calculate the predicted values
ypred = model.predict(xp)

#extract confidence invervals
_, upper,lower = wls_prediction_std(model)

#join the predicted values back to the full data
bird_combined = bird_combined.sort_values('ENSO')
ypred = pd.DataFrame({'bs_penguin_pred':ypred, 'Year':bird_combined.dropna(subset = 'bs_penguin_norm')['Year']})
bird_combined = bird_combined.merge(ypred, on = 'Year', suffixes=('', '_second_order'))
bird_combined
#add the upper and lower bounds
bird_combined['penguin_upper_ci'] = upper
bird_combined['penguin_lower_ci'] = lower

bird_combined = bird_combined.sort_values('Year')
bird_combined

In [ ]:
###Plot data with second degree polynomial
#plot the data the regression line
fig, ax = plt.subplots(1, 1, figsize = (8, 6))
ax.scatter(bird_combined['ENSO'], bird_combined['bs_penguin_norm'])

#add the regression line
ax.plot(bird_combined.sort_values('ENSO')['ENSO'], bird_combined.sort_values('ENSO')['bs_penguin_pred_second_order'], c = 'red', linestyle = 'dashed')

#add the rsquared and p-value
ax.text(0.95, 0.93, u"R\u00b2: {:0.3f}".format(r_squared), horizontalalignment = 'right', transform=plt.gca().transAxes)
ax.text(0.95, 0.89, u"p-value: {:0.3f}".format(p_val), horizontalalignment = 'right', transform=plt.gca().transAxes)

#add labels and title
ax.set(xlabel = 'ENSO Value', ylabel = 'Breeding Success', title = "ENSO Values vs Breeding Success of Brandt's Commorant")